Valid for ping 0.25 (26/10/2024)

## Data access advanced concepts

In [2]:
%matplotlib widget
import numpy as np
import themachinethatgoesping as pingmachine
import time
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

### 1. Advanced use of find_files()

In [3]:
# find_files() searches the input folder recursively (aka, including subfolders)
folder = r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all'
files = pingmachine.echosounders.index_functions.find_files(folder, [".all","wcd"])
print(len(files))

Found 476 files
476


In [4]:
# find_files() can use a list of folders as input
folders = []
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM302')
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM710')
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM2040')
files = pingmachine.echosounders.index_functions.find_files(folders, [".all","wcd"])
print(len(files))

Found 329 files
329


In [5]:
# pairs of files (e.g. .all and .wcd) don't have to be in the same folders. 
# Remember they are only paired later, by a File Handler

### 2. Advanced use of File Handler
#### 2.1 Caching 

In [6]:
# Create a File Handler to access the raw data files
# TO DO - modify to talk about caching
folder = r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM710\KV-FOSAE-2015_EM710_BH02_gas_flares'
files = pingmachine.echosounders.index_functions.find_files(folder, [".all","wcd"])

# First call to fileHandler
start_time_1 = time.time()
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(files)
end_time_1 = time.time()

# Second call to fileHandler
start_time_2 = time.time()
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(files)
end_time_2 = time.time()

print(f"\nExecution time:\n")
print(f"  First execution: {end_time_1 - start_time_1} seconds")
print(f"  Second execution: {end_time_2 - start_time_2} seconds")

Found 4 files
indexing files ⠈ 100% :00s<00m:00s] [.._092821_Karoline.wcd (4/4)]                              
indexing files ⠐ 100% :00s<00m:00s] [Found: 123491 datagrams in 4 files (2808MB)]                                             
Initializing ping interface ⢀ 50% :00s<00m:00s] [Done]                                              
indexing files ⡀ 100% :00s<00m:00s] [.._092821_Karoline.wcd (4/4)]                              
indexing files ⠄ 100% :00s<00m:00s] [Found: 123491 datagrams in 4 files (2808MB)]                                             
Initializing ping interface ⢀ 50% :00s<00m:00s] [Done]                                              

Execution time:

  First execution: 2.1386072635650635 seconds
  Second execution: 2.204694986343384 seconds


In [9]:
# First call to fileHandler, specifying the index
cache = pingmachine.echosounders.index_functions.get_cache_file_paths(file_paths=files)
start_time_1 = time.time()
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(files, file_cache_paths=cache)
end_time_1 = time.time()

# Second call to fileHandler, 
start_time_2 = time.time()
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(files, file_cache_paths=cache)
end_time_2 = time.time()

print(f"\nExecution time:\n")
print(f"  First execution: {end_time_1 - start_time_1} seconds")
print(f"  Second execution: {end_time_2 - start_time_2} seconds")

indexing files ⡀ 99% :00s<00m:00s] [Found: 123491 datagrams in 4 files (2808MB)]                                             
Initializing ping interface ⢀ 50% :00s<00m:00s] [Done]                                              
indexing files ⠄ 99% :00s<00m:00s] [Found: 123491 datagrams in 4 files (2808MB)]                                             
Initializing ping interface ⢀ 50% :00s<00m:00s] [Done]                                              

Execution time:

  First execution: 0.6172809600830078 seconds
  Second execution: 0.6140503883361816 seconds


### 3. Advanced access of Pings

In [ ]:
# pingcontainer are mostly for fast filtering of pings (eg on basis of sonar head, or location)

In [ ]:
# Access an individual ping in a Ping Container by indexing
pingContainer = fileHandler.get_pings()
ping42 = pingContainer[42]
type(ping42)

themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing

In [ ]:
#  This approach only works for a single ping. If you index a Ping Container for multiple pings, you will get a (smaller) Ping Container object
pings0To42 = pingContainer[0:42]
type(pings0To42)

themachinethatgoesping.echosounders_cppy.kongsbergall.filedatacontainers.KongsbergAllPingContainer

In [ ]:
# If you want a list of Ping objects, you need to loop over each element of the Ping Container
pings0To42 = []
for i in range(42):
    pings0To42.append(pingContainer[i])
print(f"pings0To42 is now a {type(pings0To42)} where each element is a {type(pings0To42[0])}")

pings0To42 is now a <class 'list'> where each element is a <class 'themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing'>


In [ ]:
# But you can also directly create lists of pings selected from a Ping Container by filtering for a specific critiera, for example pings containing watercolumn data
pingsWithWC = pingmachine.pingprocessing.filter_pings.by_features(pingContainer,['watercolumn.amplitudes'])
print(f"pingsWithWC is a {type(pingsWithWC)} where each element is a {type(pingsWithWC[0])}")

pingsWithWC is a <class 'list'> where each element is a <class 'themachinethatgoesping.echosounders_cppy.kongsbergall.filetypes.KongsbergAllPing'>


### 4. Access data in datagrams

In [ ]:
# Use the datagram_interface of a file handler to access the datagrams in the file
file = r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM2040\KV-Meritaito_2024_EM2040_Sorvest-F-1-1-extracts-WCD\0225_20240427_083531.all'
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(file)
print(fileHandler.datagram_interface.datagrams())

indexing file ⠐ 100% :00s<00m:00s]           
indexing file ⠠ 100% :00s<00m:00s] [Found: 1090 datagrams]                       
Initializing ping interface ⠐ 0% :00s<00m:00s] [Done]                                              
DatagramContainer
#################
-
Time info (Datagrams) 
------------------------ 
- Start time: 27/04/2024 08:35:30.79 
- End time:   27/04/2024 08:36:59.43 
- Sorted:     no                     

 Contained datagrams 
---------------------        
- Total:                                       1090 
- Datagrams [PUIDOutput]:                      2    [30]
- Datagrams [PUStatusOutput]:                  14   [31]
- Datagrams [AttitudeDatagram]:                14   [41]
- Datagrams [ClockDatagram]:                   14   [43]
- Datagrams [InstallationParametersStart]:     1    [49]
- Datagrams [RawRangeAndAngle]:                292  [4e]
- Datagrams [PositionDatagram]:                13   [50]
- Datagrams [RuntimeParameters]:               3    [52]
- Datagra

In [ ]:
# datagram_interface.datagrams() is an iterator so you can loop over all datagrams with:
for d in tqdm(fileHandler.datagram_interface.datagrams()):
    pass

# or loop over specific datagram types with
for d in tqdm(fileHandler.datagram_interface.datagrams("RuntimeParameters")):
    pass

  0%|          | 0/1090 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# index to access a specific datagram
d  = fileHandler.datagram_interface.datagrams("RuntimeParameters")[0]
print(d)

RuntimeParameters
#################
- bytes:               52       
- stx:                 0x02     
- datagram_identifier: 0x52     [RuntimeParameters]
- model_number:        EM2040   [2040]
- date:                20240427 [YYYYMMDD]
- time_since_midnight: 30931461 [ms]

 date/time 
-----------  
- timestamp: 1714.207e⁶   [s]
- date:      27/04/2024   [MM/DD/YYYY]
- time:      08:35:31.461 [HH:MM:SS]

 datagram content 
------------------             
- ping_counter:                         57744      
- system_serial_number:                 333        
- operator_station_status:              0          
- processing_unit_status:               0          
- bsp_status:                           0          
- sonar_head_or_transceiver_status:     0          
- mode:                                 0b10000001 
- filter_identifier:                    0b10100110 
- minimum_depth:                        10         [m]
- maximum_depth:                        150        [m]
- absorption_coe

In [ ]:
# and use "get_" methods to get data of interest from a datagram
print(f"Model number: {d.get_model_number()}")
print(f"Mode: {d.get_mode()}")
print(f"System serial number: {d.get_system_serial_number()}")
print(f"Transmit pulse length (raw): {d.get_transmit_pulse_length()}")
print(f"Transmit pulse length (in sec): {d.get_transmit_pulse_length_in_seconds()}")

Model number: 2040
Mode: 129
System serial number: 333
Transmit pulse length (raw): 200
Transmit pulse length (in sec): 0.00019999999494757503


In [ ]:
# Note you can also access runtime parameters applying to a given ping
ping = fileHandler.get_pings()[0]
rp = ping.file_data.get_runtime_parameters()
print(f"Transmit pulse length (in sec) for this ping: {d.get_transmit_pulse_length_in_seconds()}")

Transmit pulse length (in sec) for this ping: 0.00019999999494757503
